### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 1

Modify the entire notebook in such a way that the dispersion on the square lattice also includes next-nearest neighbor (diagonal) hopping $t^\prime$. The new dispersion relation is $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y}) - 4t^\prime \cos{k_x}\cos{k_y}$. How does the corresponding tight-binding matrix look like?

Execute the notebook for $t^\prime=-0.25t$. What are the differences in the dispersion, Fermi surface and DOS w.r.t. the case of $t^\prime$=0?

Execute the notebook for $t^\prime=-0.25t$ also with a chemical potential of $\mu=-0.5385$, which corresponds roughly to half-filling, i.e., on each lattice site sits one electron on average.

# Introduction to multivariable Green's functions

In this notebook, we learn how to create and manipulate multivariable Green's functions.
As an example, we consider the Green's function on a square lattice with nearest-neighbour hopping $t$ and next-nearest neighbor hopping $t^\prime$,

\begin{equation}
G(\mathbf{k},i\omega_n)=\frac{1}{i\omega_n  + \mu - \epsilon(\mathbf{k})}
\end{equation}

with dispersion $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y}) - 4t^\prime \cos{k_x}\cos{k_y}$. Here $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$), $\mu$ is the chemical potential and $i\omega_n$ is a Matsubara frequency.

## Imports and parameters

Below we import modules that will be useful in the following. We also set the
parameters of the problem.

In [ ]:
# Relevant Imports 
from triqs.lattice import BravaisLattice, BrillouinZone
from triqs.gf import Gf, MeshProduct, MeshBrZone, MeshImFreq
import numpy as np
from math import cos, pi

In [ ]:
# Physical parameters
beta = 5         # Inverse temperature
t    = 1.0       # Nearest-neighbor hopping (unit of energy)
tp   = -0.25 * t # Next-nearest-neighbor hopping
mu   = -0.5385   # Chemical potential

## Constructing and Initializing a Lattice Green's function

We first define a simple Bravais lattice (`BravaisLattice`) in 2 dimensions with basis vectors $\hat{e}_x = (1, 0, 0)$ and ${\hat e}_y=(0, 1, 0)$. Given this bravais lattice we construct the reciprocal (momentum) space Brillouin zone (`BrillouinZone`), on which we can then construct a momentum mesh (`MeshBrZone`).

In [ ]:
BL = BravaisLattice([(1,0,0), (0,1,0)]) # Two unit vectors in R3
BZ = BrillouinZone(BL) 

# n_k denotes the number of k-points for each dimension
n_k = 32
k_mesh = MeshBrZone(bz=BZ, n_k=n_k)

The Lattice Green's function is defined on a mesh that is the cartesian product of this momentum mesh and a Matsubara mesh.

$$
G: (\mathbf{k}, i\omega_n) \rightarrow {\mathcal{C}}
$$

To construct this mesh we use the `MeshProduct` provided by TRIQS:

In [ ]:
iw_mesh = MeshImFreq(beta=beta, statistic='Fermion', n_iw=128)
k_iw_mesh = MeshProduct(k_mesh, iw_mesh)

# Recall that for an empty target_shape G0 has values that are scalars instead of matrices.
G = Gf(mesh=k_iw_mesh, target_shape=[])

To fill the Green's function we construct a function for the dispersion $\epsilon(\mathbf{k})$ and set each element of $G$ by looping over the momentum and frequency meshes.

In [ ]:
def eps(k):
    return -2 * t * (cos(k[0]) + cos(k[1])) - 4 * tp * cos(k[0]) * cos(k[1])

In [ ]:
%%timeit
# Loop initialization. Slow..
for k, iw in G.mesh:
    G[k, iw] = 1/(iw + mu - eps(k))

## Numpy Broadcasting

Instead of writing a loop we can use the [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) features of the numpy package to assign directly into the data-array of the Green's function object. This approach is a lot faster than writing a loop

In [ ]:
iw_arr = np.array(list(iw_mesh.values()))
k_arr  = np.array(list(k_mesh.values()))
np_eps = np.vectorize(eps, signature='(d)->()')

In [ ]:
%%timeit
# Vectorized function evaluation
eps_arr = np_eps(k_arr)

# Numpy Broadcasting
G.data[:] = 1.0 / (iw_arr[None,::] + mu - eps_arr[::,None])

We provide through the [TRIQS/tprf](https://triqs.github.io/tprf) application more efficient parallelized routines for initializing lattice Green functions. Those will be introduced in the **Two Particle Reponse** Notebooks.

## Evaluate the Green function

The Green's function object $G(k,i\omega_n)$ can be evaluated like an ordinary Python function
at a given reciprocal vector and Matsubara frequency:

- The reciprocal vector $k$ is a tuple/list/numpy.array of double 
- The Matsubara frequency is an integer $n$, the $n$ in $i\omega_n$

The result will be a linear interpolation on the Brillouin zone 
  with the points on the grid of $G$ around $k$.

Therefore, one can use $g_0$ as any python function of $k$ and $i\omega_n$, 
and forget its precise representation in memory (what is the grid, etc...).
We will use that in the plot functions below.

Example:
Let's evaluate the above Green's function at $\mathbf{k} = (\pi,\pi,0)$ and $i\omega_2$. As $\epsilon((\pi,\pi,0)) = 4t + t = 5$ and $i\omega_2 = i\frac{(2*2 + 1)\pi}{\beta}$, we check:

In [ ]:
G_eval  = G((pi,pi,0), 2)
G_exact = 1.0/(1j * (2*2+1)*pi/beta - 5 + mu)
print(G_eval - G_exact) # Check

## Partial evaluation

Given a function $G(k,i\omega_n)$ it is possible to obtain the function $i\omega_n \rightarrow G(k_0, i\omega_n)$ for a fixed $k_0$:

In [ ]:
k0 = (0.02,0.01,0)      # a k-point as a tuple of 3 floats
Giw = G(k0, all)        # We use the "built-in" function all here as equivalent of :, 
                        # which Python does not allow in ()
    
# Giw is a Green's function of the Matsubara frequency only
# It is calculated by k-interpolation of G
print(Giw)

# Giw uses the original Matsubara mesh
assert Giw.mesh == G.mesh[1]

Here `Giw` is obtained through linearly of `G` for the point $k_0$ on the original Brillouin zone grid.

It is simply a Matsubara Green's function, which means that you can use all the common methods, such as `density()` or Fourier transforms:

In [ ]:
# This is the density n_k at k=(0.02, 0.01)
print("n_k =",  Giw.density().real)

## Defining a Tight-Binding Hamiltonian

In practice we often know the tight-binding Hamiltonian on our Bravais lattice rather than the analytic dispersion relation.
TRIQS provides the `TightBinding` class for this case:

In [ ]:
from triqs.lattice import TightBinding
?TightBinding

In [ ]:
# Define mapping between displacement vectors and hopping amplitudes
# Matrix structure of the amplitudes is w.r.t. atoms in the unit cell (here only one).
hop= {  (1,0)    :  [[ -t]],       
        (-1,0)   :  [[ -t]],     
        (0,1)    :  [[ -t]],
        (0,-1)   :  [[ -t]],
        (1,1)    :  [[ -tp]],
        (1,-1)   :  [[ -tp]],
        (-1,1)   :  [[ -tp]],
        (-1,-1)  :  [[ -tp]]
     }
TB = TightBinding(bl=BL, hoppings=hop)

# Green's function on the k_mesh holding the dispersion values
eps_k = TB.dispersion(k_mesh)[0]

# Initialize the lattice Green's function using Numpy Broadcasting
Gtb = G.copy()
Gtb.data[:] = 1.0 / (iw_arr[None,::] + mu - eps_k.data[::,None])

# Check Equality
assert np.linalg.norm((G - Gtb).data) < 1e-12

We note that the object `eps_k` returned by the `dispersion` function is also a triqs `Gf` object, but one that has only a momentum mesh. This illustrates nicely that the `Gf` class in TRIQS is very flexible w.r.t. the domain of definition (mesh), and can be used to store generic functions on the domain, not just Green Functions in the sense of the many-body definition.

Let us now plot the dispersion relation `eps_k` we have obtained

In [ ]:
# TOFIX!!!
# These lines should not be necessary (wrong cast in TB.dispersion?)
eps_k = Gf(mesh=k_mesh, target_shape=[])
eps_k.data[:] = TB.dispersion(k_mesh)[0].data[:]

In [ ]:
# Prepare the data
k_grid = k_arr.reshape(n_k,n_k,3)
X = k_grid[...,0]/pi
Y = k_grid[...,1]/pi
Z = eps_k.data.reshape(n_k,n_k)

# Plot the dispersion
from matplotlib import pyplot as plt
%matplotlib inline

fig  = plt.figure(dpi=110)
ax   = plt.axes(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap='coolwarm')
fig.colorbar(surf, shrink=0.5, aspect=10)

ax.set_xlabel(r"$k_x/\pi$")
ax.set_ylabel(r"$k_y/\pi$")
ax.set_title(r"$\epsilon(\mathbf{k})$")

plt.show()

Since the square lattice is two-dimensional it is possible to visualize the dispersion using a color plot.

Here is an example that plots $\epsilon(\mathbf{k})-\mu$ in the entire Brillouin zone as well as the shape of the Fermi surface at $\omega = 0$ (dotted line).

In [ ]:
k = np.linspace(-np.pi, np.pi, num=100)
kx, ky = np.meshgrid(k, k)

e_k_interp = np.vectorize(lambda kx, ky : eps_k((kx, ky, 0)).real - mu)(kx, ky)

plt.figure()

plt.pcolormesh(kx, ky, e_k_interp, rasterized=True, cmap='RdBu')
plt.colorbar().ax.set_ylabel(r'$\epsilon(\mathbf{k})-\mu$'); 

plt.contour(kx, ky, e_k_interp, levels=[0], linestyles='dotted')

plt.xlabel(r'$k_x$'); plt.ylabel(r'$k_y$');
k_ticks, k_labels = [-np.pi, 0, np.pi], [r"$-\pi$", r"0", r"$\pi$"]
plt.xticks(k_ticks, k_labels); plt.yticks(k_ticks, k_labels);
plt.axis('square');

# -- High-symmetry path G-X-M-G

pts = [
    (0., 0., r'$\Gamma$', 'w', 'bottom', 'right'), 
    (np.pi, 0., r'$X$', 'k', 'center', 'left'),
    (np.pi, np.pi, r'$M$', 'r', 'bottom', 'left'),
    ]
for x, y, label, color, va, ha in pts:
    plt.plot(x, y, 'o', color=color, clip_on=False, zorder=110)
    plt.text(x, y, label, color=color, fontsize=18, va=va, ha=ha)

X, Y, _, _, _, _ = zip(*(pts+[pts[0]]))
plt.plot(X, Y, '-m', zorder=100, lw=4);

Another useful tool of the tight-binding machinery is the calculation of the density of states (DOS). For this we can import the `dos` routine, which returns a density of states for a given `TightBinding` object.

In [ ]:
from triqs.lattice.tight_binding import dos

d = dos(TB, n_kpts=500, n_eps=101, name='DOS')[0]

In [ ]:
# Plot the dos it with matplotlib
from triqs.plot.mpl_interface import oplot

oplot(d,'-o')
plt.xlim(-5,6)
plt.xlabel(r'$\omega$')
plt.ylim(0,0.4)
plt.ylabel(r'DOS$(\omega)$')
plt.vlines(x=mu,ymin=0.,ymax=0.4, linestyle='--', color='gray', label=r'$\omega=\mu$')
plt.vlines(x=0.,ymin=0.,ymax=0.4, linestyle='-', color='black', linewidth=0.3, label=r'$\omega=0$')
plt.legend(loc=1)
plt.show()

Since this is a non-interacting problem, the DOS should be equivalent to $-\frac{1}{\pi}$Im $G^R_\text{loc}(\omega)$. We first calculate the local Green function by summing over all $\mathbf{k}$-points. We have to take a matrix-valued Green function since the next step is not (yet) implemented for a scalar-valued one.

In [ ]:
G_loc = Gf(mesh=iw_mesh, target_shape=[1,1])

for k, iw in k_iw_mesh:
    G_loc[iw] = G_loc[iw] + G[k,iw]
G_loc = G_loc / len(k_mesh)

Now we analytically continue the function with Pade.

In [ ]:
# A uniform real-frequency mesh on a given interval
from triqs.gf import MeshReFreq
w_mesh = MeshReFreq(window=(-6,6), n_w=1001)

# Construct real-frequency Green's function and initialize it using Pade approximants
G_w = Gf(mesh=w_mesh, target_shape=[1,1])
G_w.set_from_pade(G_loc)

As a last step we plot $-\frac{1}{\pi}$Im $G^R_\text{loc}(\omega)$.

In [ ]:
oplot(-G_w.imag/pi, linewidth=2, label='DOS')
plt.xlim(-5,5)
plt.xlabel(r'$\omega$')
plt.ylim(0,0.8)
plt.ylabel(r'DOS$(\omega)$')
plt.vlines(x=mu,ymin=0.,ymax=0.8, linestyle='--', color='gray', label=r'$\omega=\mu$')
plt.vlines(x=0.,ymin=0.,ymax=0.8, linestyle='-', color='black', linewidth=0.3, label=r'$\omega=0$')
plt.legend(loc=1)
plt.show()

We see that this roughly resembles the DOS. The differences may stem from the non-zero  temperature (the DOS is calculated at $T=0$), but much more it shows the potential problems of performing analytic continuation on numerical data.